# Run Lambada on the new models and compare
- This should give us an idea whether the finetuned models loose LM capacities



Results: All works reasonably well. However the LM cappacities are slightly declining.

- 0.3548 on the test set
- 0.325 on the test set when finetuned 1 Epoch
- 0.3264 on the test set when finetuned 20 Epochs
- 0.342 on test set using the full translation model trained for 20 Epochs
- 0.3142 when combining both together (finetuned generation translated) 




In [1]:
lam = open("LAMBADA/lambada_control_test_data_plain_text.txt","r+",encoding="utf-8")
lam = lam.read()
lam = lam.splitlines()

In [2]:
len(lam)

5000

In [3]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

In [4]:
import torch
import pickle
from utility import * 

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [6]:
def finetuned(path):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.load_state_dict(torch.load("../transformers/examples/model_save/" + str(path)))
    return model

In [11]:
%%capture
lamT = []
for x in lam:
    lamT.append(tokenizer.encode(x))

# Baseline

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [9]:
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(GPT_org(torch.tensor(x[:-1]))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds.p","wb"))

In [15]:
predictions = pickle.load(open("saves/save_preds.p","rb"))

In [16]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.3548


# Normal finetune

- 20 Epochs
- 1 Epoch

In [9]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_classic_finetune/checkpoint-41599/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(model(torch.tensor(x[:-1]).to("cuda"))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds_fine_1.p","wb"))

In [19]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream_classic_finetune/pytorch_model.bin"))
model = model.to("cuda")

In [ ]:
predictions = []
count = 0
for x in lamT:
    count +=1
    if count%100 == 0:
        print(count)
    prediction = tokenizer.decode(torch.argmax(model(torch.tensor(x[:-1]).to("cuda"))[0][-1:,:]).tolist())
    real = tokenizer.decode(x[-1])
    results = (prediction,real)
    predictions.append(results)
pickle.dump(predictions,open("saves/save_preds_fine_20.p","wb"))

In [21]:
predictions = pickle.load(open("saves/save_preds_fine_1.p","rb"))

In [22]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.325


In [24]:
predictions = pickle.load(open("saves/save_preds_fine_20.p","rb"))

In [25]:
counter = 0
for occasion in predictions:
    if occasion[0] == occasion[1]:
        counter+=1
print(counter/len(predictions))

0.3264


# Translation model fully trained

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [8]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

In [12]:
step1 = pickle.load(open("saves/save_preds.p","rb"))
step1_corpus = lamT

In [13]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [14]:
eos = split_eos(out)

In [23]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream/pytorch_model.bin"))
model = model.to("cuda")

In [24]:
%%capture
predictions = []
for x in eos: 
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_trans.p","wb"))

In [26]:
prediction = pickle.load(open("saves/save_preds_trans.p","rb"))

In [28]:
real = pickle.load(open("saves/save_preds.p","rb"))

In [29]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [30]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [32]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.3418


# Combining both

In [33]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [34]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

In [35]:
step1 = pickle.load(open("saves/save_preds_fine_1.p","rb"))
step1_corpus = lamT

In [36]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [37]:
eos = split_eos(out)

In [39]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("trained_models/the_dream/pytorch_model.bin"))
model = model.to("cuda")

In [40]:
%%capture
predictions = []
for x in eos: 
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("saves/save_preds_combined.p","wb"))

In [41]:
prediction = pickle.load(open("saves/save_preds_combined.p","rb"))
real = pickle.load(open("saves/save_preds.p","rb"))

In [42]:
out = []
for x in prediction:
    out.append(x.replace("<|endoftext|>",""))
prediction = out

In [43]:
%%capture
pred = []
for x in prediction: 
    pred.append(tokenizer.decode(tokenizer.encode(x)[-1]))

In [44]:
counter = 0
for x in range(len(pred)):
    if pred[x] == real[x][1]:
        counter+=1
print(counter/len(pred))

0.314
